In [23]:
#Part II

#Data Preparation (From Part I)

import pandas as pd
import numpy as np

def parse_data(filename):
    A=[]
    text_file = open(filename, "r")
    lines = text_file.readlines()
    text_file.close()
    for i in range(0,len(lines)):
        id = lines[i].split(' ', 1)[0]
        string = lines[i].split(' ', 1)[1]
        rem_string = ''.join(e for e in string if e.isalnum()).lower()
        tupool = (id,rem_string)
        A.append(tupool)
    return A


In [24]:
#Shingle (From Part I)
import binascii

def shingle_document(string, k):
    word=string
    ShinglesHash = set()
    shingles = [word[i:i+k] for i in range(0, len(word)-k+1) if len(word[i])<k+1]
    for i in shingles:
        crc = binascii.crc32(str.encode(i)) & 0xffffffff
        ShinglesHash.add(crc)
    return ShinglesHash
    

In [25]:
#Jaccard (From Part I)
def jaccard(a, b):
    
    intersectionsize=len(a.intersection(b))
    unionsize=len(a.union(b))
    JS=intersectionsize/unionsize
    return JS
    

In [26]:
#Part II

#Part IIA

def invert_shingles(shingled_documents):
    invert_item = []
    aIDs = []
    
    for (aID, shingles) in shingled_documents:
        aIDs.append(aID)
        for s in shingles:
            item = (s, aID)
            invert_item.append(item)
    results=sorted(invert_item),aIDs
    return results


In [27]:
#Part IIB

#Make Random Hashes

import random

def make_random_hash_fn(p=2**33-355, m=4294967295):
    a = random.randint(1,p-1)
    b = random.randint(0, p-1)
    return lambda x: ((a * x + b) % p) % m


In [40]:
#Part IIC
import numpy

def make_minhash_signature(shingled_data, num_hashes):
    make_hashes=make_random_hash_fn()
    inv_index, docids = invert_shingles(shingled_data)
    num_docs = len(docids)
  
  # initialize the signature matrix with infinity in every entry
    sigmatrix = np.full([num_hashes, num_docs], np.inf)
  
  # generate hash functions
    hash_funcs = make_hashes(num_hashes)
  
  # iterate over each non-zero entry of the characteristic matrix
    for row, docid in inv_index:
    # update signature matrix if needed 
        hashvals=[h(row) for h in hash_funcs]
        j=docids.index(docid)
    
        for i in range(num_hashes):
            if hashvals[i]<sigmatrix[i,j]:
                sigmatrix[i,j]=hashvals[i]
    # THIS IS WHAT YOU NEED TO IMPLEMENT
  
    return sigmatrix, docids




In [44]:
#Part IIC

def minhash_similarity(id1, id2, minhash_sigmat, docids):
    
    
    # get column of the similarity matrix for the two documents
    i=docids.index(id1)
    j=docids.index(id2)
    a=minhash_sigmat[:,i]
    b=minhash_sigmat[:,j]
            
            # calculate the fraction of rows where two columns match
    fraction = jaccard(a,b)
    
           
    # return this fraction as the minhash similarity estimate
    return fraction
  



In [49]:
#Part IID

import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plot

Location=r'C:\Users\yazdan\Desktop\MinHash-master\MinHash-master\data\articles_1000.train'

DATA = parse_data(Location)
toople=[]
results=[]
num_hashes=[10, 20, 30, 40, 50 ,60, 70, 80, 90, 100]
rmse=[]
k=10

for i in range(0,len(DATA)):
    a=shingle_document(DATA[i][1],10)
    aID=DATA[i][0]
    toople.append((aID,a))

for N in num_hashes:

    sigmat,docids=make_minhash_signature(toople,num_hashes)

    for i in range(0,len(DATA)):
        for j in range(i+1,len(DATA)):

            id1=DATA[i][0]
            id2=DATA[j][0]
            MS=minhash_similarity(id1, id2,sigmat, docids) # Calling MinHash

            a=shingle_document(DATA[i][1],k)
            b=shingle_document(DATA[j][1],k)
            JS=jaccard(a,b)       # Calling Jaccard

            results.append((id1,id2,MS,JS))   #Table of Results

    # RMSE Error

    rmse.append = sqrt(mean_squared_error(JS, MS))
    

plot.scatter(num_hashes,rmse, label='Error Plot')    
plot.legend()
plot.xlabel('Num_Hashes')
plot.ylabel('RMSE')
plot.show()

TypeError: 'list' object cannot be interpreted as an integer